# Participant Data analysis 


### Some participant ID may not be either CI or NH, find who is neither. 
- review how many participants in .txt ifle have a group designation in latest_demo.csv(CI vs. NH) in the data folder. 
### Crate table, how many words were produced by the NH and CI groups (get average of participants in each group) 

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
latest_demo = pd.read_csv("Raw Data/latest_demo.csv")
latest_demo

In [ ]:
data_cochlear = pd.read_csv("Raw Data/data-cochlear.txt", delimiter= "\t")
words = data_cochlear["Word"].tolist()
data_cochlear



### How many participants in data-cochlear.txt have a group designation in latest_demo.csv


In [ ]:
participants_findings = pd.DataFrame()


#Unique IDs in data-cochlear.txt
participants = sorted(list(set(data_cochlear["ID"].tolist())))
print(participants)

participants_findings["Original ID"] = participants


#Participants list with only numbers. 
modified_participants = []
for part in participants: 
    modified_participants += [int(part[4:])]
    
participants_findings["Modified ID"] = modified_participants
#getting the number of responses by the participant ID. 



#Getting the groups for data-cochlear

#iterating and checking the 
groups = []
for id in modified_participants: 
    target = id
    
    condition = latest_demo["ID"] == id 
    
    if condition.any():
        group = latest_demo.loc[condition, 'Group'].values[0]
        groups += [group] 
    else: 
        groups += ["no group"]
print(groups)
partcipants_dict = {participants[i]: groups[i] for i in range(len(participants))}
print(partcipants_dict.items())


participants_findings["Group"] = groups
participants_findings.to_csv("Findings/participants_findings.csv")

### regroup the participants into three groups: without_group, NH, CI 

In [ ]:
#Number of individuals without a group designation 

without_group = []
NH = []
CI = [] 
i = 0 
while i < len(participants):
    if groups[i] == "no group":
        without_group += [participants[i]]
    if groups[i] == "NH": 
        NH += [participants[i]]
    if groups[i] == "CI":
        CI += [participants[i]]
    i += 1 
    
print(without_group)
print(len(without_group))
print("--------------------")

print(NH)
print(len(NH))
print("-------------")

print(CI) 
print(len(CI))






### Separating the groups and creating 3 txt files for without_groups, NH, CI 


In [ ]:
with open("Raw Data/data-cochlear.txt", "r") as contents: 
    first = contents.readline()
    lines = contents.readlines()
    

without_groups_data =[]
NH_data = []
CI_data = []


for line in lines: 
    if line[0:7] in without_group: 
        without_groups_data += [line]
    elif line[0:7] in NH: 
        NH_data += [line] 
    elif line[0:7] in CI: 
        CI_data += [line]

with open("Participant Data/NO-ID-cochlear-data.txt", "w") as contents: 
    contents.write("ID\tWord\n")
    for item in without_groups_data: 
        contents.write(item)
        


with open("Participant Data/NH-cochlear-data.txt", "w") as f: 
    f.write("ID\tWord\n")
    for item in NH_data: 
        f.write(item)


with open("Participant Data/CI-cochlear-data.txt", "w") as c: 
    c.write("ID\tWord\n")
    for item in CI_data: 
        c.write(item)
        
print(without_groups_data)
print(NH_data)
print(CI_data)

# created separate data for no group, nh, and ci 


### Counting the number of words produced from NH and CI 

In [ ]:
NH_words = pd.read_csv("Participant Data/NH-cochlear-data.txt", delimiter="\t")
CI_words = pd.read_csv("Participant Data/CI-cochlear-data.txt", delimiter="\t")

# NH_words.describe()
# CI_words.describe()



NH_dict = {NH[i]: 0 for i in range(len(NH))}
CI_dict = {CI[i]: 0 for i in range(len(CI))}

for line in NH_data: 
    NH_dict[line[0:7]] += 1

for line in CI_data:
    CI_dict[line[0:7]] += 1 


word_findings = pd.DataFrame(list(zip(NH_dict.keys(), NH_dict.values(), CI_dict.keys(), CI_dict.values())), columns=['NH ID', 'NH Word Count', 'CI ID', 'CI Word Count'])

word_findings.to_csv("Findings/word_findings.csv")



## Creating a Modified participant data with replacments (ex. chimp -> chimpanzee -> primate)

In [4]:
import pandas as pd 

data = pd.read_csv("Participant Data/data-cochlear.txt",delimiter="\t")
replacements = pd.read_csv("Findings/data-cochlear-replacements.csv")
NO_ID = ['CAN-667', 'CAU-667', 'SFZ-149', 'SOA-317']
for id in NO_ID: 
    data = data.drop(data[data['ID'] == id].index)
    replacements = replacements.drop(replacements[replacements['ID'] == id].index)

replacements = replacements.drop(replacements[replacements['Word'] == 'stickforn'].index)
replacements = replacements.drop(replacements[replacements['Word'] == 'UNKNOWN'].index)

replacements.to_csv("Findings/data-cochlear-replacements.csv")

id = data["ID"].tolist()
words = data['Word'].tolist() 

modified = replacements['Modified'].tolist()

in_s2v = replacements['In W2V/S2v'].tolist()

replacement = replacements['chosen_replacement'].tolist()

i = 0 
while i < len(words): 
    if in_s2v[i] == 'no': 
        modified[i] = replacement[i]
    i += 1
new_participant_data = pd.DataFrame(data= list(zip(id, modified)), columns = ['ID', 'Word'])
new_participant_data.to_csv("Participant Data/new_data.csv", index = False)